In [1]:
import sqlite3
connection = sqlite3.connect('Data/chinook.db')
cursor = connection.cursor()
cursor.execute('SELECT * FROM artists LIMIT 5;')
cursor.fetchall()

[(1, 'AC/DC'),
 (2, 'Accept'),
 (3, 'Aerosmith'),
 (4, 'Alanis Morissette'),
 (5, 'Alice In Chains')]

In [ ]:
query = """
SELECT *
FROM artists
LIMIT 5;
"""
cursor.execute(query)
cursor.fetchall()